In [1]:
import tensorflow as tf
import os
from pathlib import Path
import urllib.request as request
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = (224,224)

In [ ]:
model = tf.keras.applications.VGG16(
    include_top=True,weights='imagenet',input_tensor=None,input_shape=None,classes=1000

)

In [ ]:
model.summary()

In [ ]:
model.save('VGG16_full_model.h5')

In [ ]:
model.layers[1].padding

In [ ]:
data_URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

# create a directory
def create_dirs(dir_path):
    os.makedirs(dir_path, exist_ok=True)
    print(f"{dir_path} directory created")
    
ROOT_DATA_DIR = "data"
create_dirs(ROOT_DATA_DIR)


data_zip_file = "data.zip"
data_zip_path = os.path.join(ROOT_DATA_DIR, data_zip_file)

if not os.path.isfile(data_zip_file):
    print("downloading data...")
    filename, headers = request.urlretrieve(data_URL, data_zip_path)
    print(f"filename: {filename} created with info \n{headers}")
else:
    print(f"file is already present")

In [ ]:
from zipfile import ZipFile

unzip_data_dirname = "unzip_data_dir"
unzip_data_dir = os.path.join(ROOT_DATA_DIR, unzip_data_dirname)

if not os.path.exists(unzip_data_dir):
    os.makedirs(unzip_data_dir, exist_ok=True)
    with ZipFile(data_zip_path) as f:
        f.extractall(unzip_data_dir)
else:
    print(f"data already extacted")

In [ ]:
main_data_dir = Path("./data/unzip_data_dir/hymenoptera_data/train")
main_data_dir

In [ ]:
BATCH_SIZE = 32
pixels = 224
IMAGE_SIZE = (pixels, pixels)

In [ ]:
datagen_kwargs = dict(rescale=1./255, validation_split=0.20)

dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)

In [ ]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)


valid_generator = valid_datagen.flow_from_directory(
    main_data_dir, subset="validation", shuffle=False, **dataflow_kwargs)


do_data_augmentation = False

if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotational_range=40,
        horizontal_flip=True,
        width_shift_range=0.2, 
        height_shift_range=0.2, 
        shear_range=0.2, 
        zoom_range=0.2, 
        **datagen_kwargs)
else:
    train_datagen = valid_datagen

train_generator = train_datagen.flow_from_directory(
    main_data_dir, subset="training", shuffle=True, **dataflow_kwargs
)

In [ ]:
RGB_IMAGE_SIZE = (pixels, pixels, 3)


vgg = tf.keras.applications.vgg16.VGG16(
    input_shape=RGB_IMAGE_SIZE,
    weights="imagenet",
    include_top=False
)

In [ ]:
vgg.summary()

In [ ]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
for layer in vgg.layers:
    print(f"{layer.name:20s}: {layer.trainable}")

In [ ]:
vgg.output


In [ ]:
del model

In [ ]:
CLASSES = 2
x = tf.keras.layers.Flatten()(vgg.output)
prediction = tf.keras.layers.Dense(CLASSES, activation="softmax")(x)

model = tf.keras.models.Model(inputs=vgg.input, outputs = prediction)
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [ ]:
EPOCHS = 10

history = model.fit(
    train_generator, epochs=EPOCHS, validation_data=valid_generator
)

In [ ]:
model.save("trained_model.h5")

In [ ]:
label_map = {val: key for key, val in train_generator.class_indices.items()}
label_map

In [ ]:
test_img = plt.imread("./data/unzip_data_dir/hymenoptera_data/val/ants/800px-Meat_eater_ant_qeen_excavating_hole.jpg")
plt.imshow(test_img)

In [ ]:
test_img.shape


In [ ]:
model.predict(test_img)


In [ ]:
tf.image.resize(test_img, (224, 224)).shape


In [ ]:
resized_img = tf.image.resize(test_img, (224, 224))


In [ ]:
resized_img = tf.image.resize(test_img, (224, 224))


In [ ]:
input_data = tf.expand_dims(resized_img, axis=0)


In [ ]:
pred = model.predict(input_data)
pred

In [ ]:
argmax = tf.argmax(pred[0]).numpy()


In [ ]:
label_map[argmax]


In [ ]:
loaded_model = tf.keras.models.load_model("./trained_model.h5")


In [ ]:
preprocessd_img= tf.keras.applications.vgg16.preprocess_input(resized_img)


In [ ]:
preprocessd_img.shape


In [ ]:
plt.imshow(preprocessd_img)


In [ ]:
plt.imshow(test_img)


In [ ]:
full_model = 138_357_544
our_model = 14_714_688

In [ ]:
type(full_model)


In [ ]:
our_model/full_model
